In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmc-ml-steam-activity-prediction/sample_submission.csv
/kaggle/input/cmc-ml-steam-activity-prediction/users.csv
/kaggle/input/cmc-ml-steam-activity-prediction/games_details.csv
/kaggle/input/cmc-ml-steam-activity-prediction/users_friends.csv
/kaggle/input/cmc-ml-steam-activity-prediction/simple_pipeline.ipynb
/kaggle/input/cmc-ml-steam-activity-prediction/achievements_stats.csv
/kaggle/input/cmc-ml-steam-activity-prediction/train.csv
/kaggle/input/cmc-ml-steam-activity-prediction/test.csv


In [2]:
# Используйте эти пути для запуска ноутбука на Kaggle
from sklearn.metrics import mean_squared_log_error

PATH_TO_KAGGLE_TRAIN = "/kaggle/input/cmc-ml-steam-activity-prediction/train.csv"
PATH_TO_KAGGLE_TEST = "/kaggle/input/cmc-ml-steam-activity-prediction/test.csv"
PATH_TO_KAGGLE_SUBMISSION = "/kaggle/working/submission.csv"

# Если запускаете на своем компьютере, то впишите актуальные пути
# PATH_TO_TRAIN = # Your Path
# PATH_TO_TEST = # Your Path
# PATH_TO_SUBMISSION = # Your Path

train = pd.read_csv(PATH_TO_KAGGLE_TRAIN)
test = pd.read_csv(PATH_TO_KAGGLE_TEST)

data_train = pd.read_csv(PATH_TO_KAGGLE_TRAIN).drop(['index'], axis=1)

data_train.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,user_id,game_id,game_name,playtime_forever,playtime_2weeks
0,4RPRA2QQRR,971680,Montaro : RE,57,0
1,KPCXI0AMMF,527760,Just Hero,210,0
2,VJU71I4Z4D,361280,Turmoil,861,0
3,TDQ0HDZ78A,578080,PUBG: BATTLEGROUNDS,16774,0
4,31D0FURI91,311080,Echelon,198,0


In [3]:
details = pd.read_csv("/kaggle/input/cmc-ml-steam-activity-prediction/games_details.csv")
users = pd.read_csv("/kaggle/input/cmc-ml-steam-activity-prediction/users.csv")


In [4]:
data_train = data_train.merge(details, on='game_id', how='left').merge(users, on='user_id', how='left')

In [5]:
data_train.head()

,user_id,game_id,game_name,playtime_forever,playtime_2weeks,Unnamed: 0_x,genres,tags,release_date,rating,Unnamed: 0_y,communityvisibilitystate,profilestate,personastate,timecreated
0,4RPRA2QQRR,971680,Montaro : RE,57,0,1685.0,"['Casual', 'Indie']","['Singleplayer', 'Steam Achievements', '2D']",2018-11-16,0.00,6432,3,1.0,0,2006-03-21 13:07:38
1,KPCXI0AMMF,527760,Just Hero,210,0,1492.0,"['Casual', 'Indie']","['Singleplayer', 'Steam Achievements', 'Full c...",2016-09-28,0.00,654,3,1.0,1,2007-04-21 13:22:26
2,VJU71I4Z4D,361280,Turmoil,861,0,13321.0,"['Strategy', 'Indie', 'Simulation']","['Singleplayer', 'Steam Achievements', 'Steam ...",2016-06-01,3.45,6719,3,1.0,0,2012-07-04 07:44:53
3,TDQ0HDZ78A,578080,PUBG: BATTLEGROUNDS,16774,0,11246.0,"['Shooter', 'Massively Multiplayer', 'Action']","['Multiplayer', 'Co-op', 'Open World', 'cooper...",2017-12-20,3.29,7267,3,1.0,3,2009-03-27 12:31:13
4,31D0FURI91,311080,Echelon,198,0,1340.0,"['Action', 'Simulation']","['Singleplayer', 'steam-trading-cards']",2001-03-21,3.50,2743,3,1.0,0,2014-02-02 07:17:29


In [6]:
data_train.drop(['user_id', 'game_name', 'genres', 'tags', 'release_date', 'Unnamed: 0_x', 'Unnamed: 0_y', 'timecreated', 'game_id'], axis=1, inplace=True)

In [7]:
data_train

,playtime_forever,playtime_2weeks,rating,communityvisibilitystate,profilestate,personastate
0,57,0,0.00,3,1.0,0
1,210,0,0.00,3,1.0,1
2,861,0,3.45,3,1.0,0
3,16774,0,3.29,3,1.0,3
4,198,0,3.50,3,1.0,0
...,...,...,...,...,...,...
129995,153,0,2.80,3,1.0,0
129996,2107,0,0.00,3,1.0,3
129997,6,0,0.00,3,1.0,0
129998,4454,0,4.16,3,1.0,0


In [8]:
X_train = data_train.drop(['playtime_forever'], axis=1)
y_train = data_train.playtime_forever

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [10]:
y_train = np.log(y_train + 1)

In [11]:
model = GradientBoostingRegressor(n_estimators=2000, max_depth=10)

In [12]:
X_train.fillna(0, inplace=True)


In [13]:
model.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=10, n_estimators=2000)

In [14]:
y_pred = model.predict(X_train)

In [15]:
y_pred[y_pred < 0] = 0

In [16]:
np.sqrt(mean_squared_log_error(np.exp(y_train) - 1, np.exp(y_pred) - 1))

2.2616254027670073

In [17]:
data_test = pd.read_csv(PATH_TO_KAGGLE_TEST)

In [18]:
data_test = data_test.merge(details, on='game_id', how='left').merge(users, on='user_id', how='left')

In [19]:
data_test.drop(['user_id', 'game_name', 'genres', 'tags', 'release_date', 'Unnamed: 0_x', 'Unnamed: 0_y', 'timecreated', 'game_id', 'index'], axis=1, inplace=True)

In [20]:
data_test.head()

,playtime_2weeks,rating,communityvisibilitystate,profilestate,personastate
0,0,0.00,3,1.0,0
1,1074,3.83,3,1.0,1
2,0,3.70,3,1.0,0
3,0,3.74,3,1.0,0
4,0,0.00,3,1.0,0


In [21]:
X_test = data_test
X_test.fillna(0, inplace=True)


In [22]:
y_pred = model.predict(X_test)
y_pred[y_pred < 0] = 0
y_pred = np.exp(y_pred) - 1

In [23]:

submission = pd.DataFrame({"index": test["index"], 
                           "playtime_forever": y_pred})

In [24]:
submission.to_csv(PATH_TO_KAGGLE_SUBMISSION, index=False)
